# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['AngleX', 'AngleY', 'LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [14]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [15]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Stand', 'Left', 'Crouch', 'Sit', 'Belly', 'Knee', 'StandInit', 'Back', 'Right', 'Frog', 'HeadBack']


In [16]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    print len(data), len(target)
    return data, target

In [17]:
# load all the data
all_data = []
all_target = []

for i in range(0,len(classes)):
    data,target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
    
    
print 'total number of data', len(all_data)

11 11
20 20
30 30
26 26
10 10
10 10
52 52
10 10
11 11
10 10
10 10
total number of data 200


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [18]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

In [19]:
print training_data

[121 140 141   1 123  38 164 172  94 194  86 153  73  82 136  52 180  54
  72  56 143  64  57  46 126  63 124  12 138  45  23 129 125 139  35 157
  10  47  28  24 176  32  91  22  88 177  36 103 132 197  21  31 150  87
 178  42 104  65   0 160  43  15  74 190 193  37  60 148  70 198 122  85
  25 174 181 158  58 179 182 171  59  75  67 173  62 101 108 106 184  76
 168 162  39  29   7 113  95 145  69  16 169 195  18 133 135 167  97   3
  26 111  77 163 196 192  40 137 100 128 107  71  78  41  81 130 116  13
   8   5 118  99  79 114  44 187 105 189  30  17 142 151]


In [20]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [21]:
fit_data = []
fit_target = []
for i in range(0,len(training_data)):
    fit_data.append(all_data[training_data[i]])
    fit_target.append(all_target[training_data[i]])
    
clf.fit(fit_data, fit_target) 

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

### predicting

In [22]:
clf.predict(all_data[1]), all_target[1]

(array([0]), 0)

In [23]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

## 4. Evaluate on the test data

In [24]:
expected = []
predicted = []
prediction_data = []
for i in range(0,len(all_data)):
    if( not (i in training_data)):
        expected.append(all_target[i])
        prediction_data.append(all_data[i])
predicted = clf.predict(prediction_data)

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         4
          1       1.00      1.00      1.00         5
          2       1.00      1.00      1.00         8
          3       1.00      1.00      1.00         6
          4       1.00      1.00      1.00         5
          5       1.00      1.00      1.00         2
          6       1.00      1.00      1.00        18
          7       1.00      1.00      1.00         4
          8       1.00      1.00      1.00         2
          9       1.00      1.00      1.00         4
         10       1.00      1.00      1.00         2

avg / total       1.00      1.00      1.00        60


Confusion matrix:
[[ 4  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  0  0  0  0]
 [ 0  0  0  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [25]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [26]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
for i in range(0,len(all_data),5):
    print classes[clf2.predict(all_data[i])], classes[all_target[i]]

Stand Stand
Stand Stand
Stand Stand
Left Left
Left Left
Left Left
Left Left
Crouch Crouch
Crouch Crouch
Crouch Crouch
Crouch Crouch
Crouch Crouch
Crouch Crouch
Sit Sit
Sit Sit
Sit Sit
Sit Sit
Sit Sit
Belly Belly
Belly Belly
Knee Knee
Knee Knee
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
StandInit StandInit
Back Back
Back Back
Right Right
Right Right
Frog Frog
Frog Frog
HeadBack HeadBack
HeadBack HeadBack


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  app.launch_new_instance()
